In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.impute import KNNImputer
from sklearn.linear_model import LassoLars, LassoLarsCV, LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV
import matplotlib.cm as cm

warnings.filterwarnings("ignore")

# Set random seed for reproducibility
enable_halving_search_cv
np.random.seed(42)

## Sub-task 1.

The file "domations.csv" contains a sample of information about participants in the program of
donating money to the needs of veterans' organizations. Each record is one person from the
mailing list. He has socio-demographic characteristics (gender, age, median
income in the area of ​​his residence, whether he is a homeowner, etc., etc.),
behavioral characteristics (aggregated characteristics of his early donations such as
GiftCount36 - the number of donations over three years, GiftAmntLast - the amount of the last
donation, PromCntCard12 - the number of contacts with him over the year as part of an advertising campaign,
etc.). There are also two responses: the TargetB flag (whether he donated or not) and TargetD - the
donation amount (skip if he did not donate, otherwise the amount in dollars). As part of the first part
of the task, you need to build regression models (only for people who donated money),
explaining and predicting the donation amount TargetD

In [ ]:
# Load the donations dataset
data = pd.read_csv("../data/donations.csv")

# Display basic information about the dataset
print("Dataset Shape:", data.shape)
print("\nData Types:")
print(data.dtypes)

# Summary statistics
print("\nSummary Statistics:")
print(data.describe())

# Check for missing values
print("\nMissing Values per Column:")
print(data.isnull().sum())

# Let's see the first few rows
print("\nFirst 5 rows:")
print(data.head())

# Since we're only interested in records where people donated,
# filter the dataset for records where TargetB == 1 (donated)
donors = data[data["TargetB"] == 1].copy()
print(f"\nNumber of donors: {donors.shape[0]}")
print(f"Number of non-donors: {data[data['TargetB'] == 0].shape[0]}")

# Check the distribution of the target variable (TargetD - donation amount)
plt.figure(figsize=(10, 6))
sns.histplot(donors["TargetD"], kde=True)
plt.title("Distribution of Donation Amounts (TargetD)")
plt.xlabel("Donation Amount ($)")
plt.ylabel("Frequency")
plt.grid(True, alpha=0.3)
plt.show()

# Check if there are any missing values in the donation amount column
print(f"\nMissing values in TargetD for donors: {donors['TargetD'].isnull().sum()}")

# Sub-task 2.

Select and save as a holdout 30% of the original sample stratified by response. Note that the response is continuous and needs to be discretized. Choose the number of intervals and the discretization method yourself. Build and visualize a histogram (or kde approximation) for the response distribution in the entire original set, in the holdout, and in the training samples.

In [ ]:
# First, discretize the continuous response variable (TargetD) for stratification
# We'll use 5 bins (quantiles) for discretization
donors["TargetD_bins"] = pd.qcut(donors["TargetD"], q=5, labels=False)

# Split the data into train and holdout sets (70% train, 30% holdout)
train_data, holdout_data = train_test_split(
    donors,
    test_size=0.3,
    random_state=42,
    stratify=donors["TargetD_bins"],  # Stratify by the binned donation amount
)

print(f"Training set size: {train_data.shape[0]} records")
print(f"Holdout set size: {holdout_data.shape[0]} records")

# Visualize the distribution of TargetD in the original, training, and holdout sets
plt.figure(figsize=(15, 5))

# Plot for the entire dataset
plt.subplot(1, 3, 1)
sns.histplot(donors["TargetD"], kde=True, color="blue")
plt.title("Donation Distribution - Full Dataset")
plt.xlabel("Donation Amount ($)")
plt.grid(True, alpha=0.3)

# Plot for the training set
plt.subplot(1, 3, 2)
sns.histplot(train_data["TargetD"], kde=True, color="green")
plt.title("Donation Distribution - Training Set")
plt.xlabel("Donation Amount ($)")
plt.grid(True, alpha=0.3)

# Plot for the holdout set
plt.subplot(1, 3, 3)
sns.histplot(holdout_data["TargetD"], kde=True, color="red")
plt.title("Donation Distribution - Holdout Set")
plt.xlabel("Donation Amount ($)")
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Let's also compare KDE approximations
plt.figure(figsize=(10, 6))
sns.kdeplot(donors["TargetD"], label="Full Dataset", color="blue")
sns.kdeplot(train_data["TargetD"], label="Training Set", color="green")
sns.kdeplot(holdout_data["TargetD"], label="Holdout Set", color="red")
plt.title("KDE Approximation of Donation Distributions")
plt.xlabel("Donation Amount ($)")
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()

## Sub-task 3.

At the data preprocessing stage, perform missing imputation using the KnnImputer (neighbors=7) method, preserving binary features about which variables were imputed. Transformations of categorical variables using WOE, Target encoding, Threshold
encoding and other methods, as well as transformation of numerical variables (to obtain more symmetrical distributions using log or Box-Cox) are encouraged, but not required.

In [ ]:
# Define feature types
numeric_features = [
    col for col in train_data.columns if train_data[col].dtype in ["int64", "float64"]
]
numeric_features = [
    col
    for col in numeric_features
    if col not in ["TargetB", "TargetD", "TargetD_bins", "ID"]
]

categorical_features = [
    col for col in train_data.columns if train_data[col].dtype == "object"
]

print(f"Number of numeric features: {len(numeric_features)}")
print(f"Number of categorical features: {len(categorical_features)}")

# 3a. Missing Value Imputation with KnnImputer and creating binary flags
# Create binary indicators for missing values before imputation
for feature in numeric_features + categorical_features:
    if train_data[feature].isnull().sum() > 0:
        train_data[f"{feature}_missing"] = train_data[feature].isnull().astype(int)
        holdout_data[f"{feature}_missing"] = holdout_data[feature].isnull().astype(int)

# Impute missing values in numeric features using KnnImputer
# First, let's separate the target variable and keep track of it
X_train = train_data[numeric_features].copy()
y_train = train_data["TargetD"].copy()
X_holdout = holdout_data[numeric_features].copy()
y_holdout = holdout_data["TargetD"].copy()

# Initialize and fit the KNN imputer
imputer = KNNImputer(n_neighbors=7)
X_train_imputed = imputer.fit_transform(X_train)
X_holdout_imputed = imputer.transform(X_holdout)

# Convert back to DataFrames
X_train_imputed = pd.DataFrame(
    X_train_imputed, columns=numeric_features, index=X_train.index
)
X_holdout_imputed = pd.DataFrame(
    X_holdout_imputed, columns=numeric_features, index=X_holdout.index
)

# Merge the imputed numeric features with the binary missing indicators and categorical features
for col in X_train_imputed.columns:
    train_data[col] = X_train_imputed[col]
    holdout_data[col] = X_holdout_imputed[col]

# 3b. Transform numeric variables to make them more symmetrical
# Check for skewness in numeric features
skewed_features = []
for feature in numeric_features:
    skewness = train_data[feature].skew()
    if abs(skewness) > 1:  # Threshold for considering a feature as skewed
        skewed_features.append(feature)

print(f"Number of skewed numeric features: {len(skewed_features)}")

# Apply Box-Cox transformation to skewed features
for feature in skewed_features:
    # Make sure all values are positive for Box-Cox
    if min(train_data[feature]) <= 0:
        train_data[f"{feature}_bc"] = np.log1p(
            train_data[feature] - min(train_data[feature]) + 1
        )
        holdout_data[f"{feature}_bc"] = np.log1p(
            holdout_data[feature] - min(train_data[feature]) + 1
        )
    else:
        # Use PowerTransformer with method='box-cox'
        pt = PowerTransformer(method="box-cox")
        train_data[f"{feature}_bc"] = pt.fit_transform(train_data[[feature]])
        holdout_data[f"{feature}_bc"] = pt.transform(holdout_data[[feature]])

# 3c. Target Encoding for categorical variables
# For each categorical feature, replace the category with the mean of the target variable for that category
for feature in categorical_features:
    # Dictionary to map categories to their average target value
    target_means = train_data.groupby(feature)["TargetD"].mean().to_dict()

    # Apply the encoding
    train_data[f"{feature}_target_enc"] = train_data[feature].map(target_means)
    holdout_data[f"{feature}_target_enc"] = holdout_data[feature].map(target_means)

    # Handle new categories in the holdout set by using the overall mean
    holdout_data[f"{feature}_target_enc"].fillna(
        train_data["TargetD"].mean(), inplace=True
    )

# 3d. Threshold Encoding for categorical variables
# For each categorical feature, create a binary variable indicating if the category is above a certain threshold
for feature in categorical_features:
    # Get the mean target value for each category
    category_means = train_data.groupby(feature)["TargetD"].mean()

    # Define the threshold as the median of the category means
    threshold = category_means.median()

    # Create a binary variable where 1 means the category's mean target is above the threshold
    train_data[f"{feature}_threshold_enc"] = train_data[feature].map(
        category_means.gt(threshold).astype(int)
    )
    holdout_data[f"{feature}_threshold_enc"] = holdout_data[feature].map(
        category_means.gt(threshold).astype(int)
    )

    # Handle new categories in the holdout set
    holdout_data[f"{feature}_threshold_enc"].fillna(0, inplace=True)

# Now we have preprocessed our datasets with imputation, binary flags for missing values,
# and transformed numeric and categorical features
print("\nPreprocessed Training Dataset Shape:", train_data.shape)
print("Preprocessed Holdout Dataset Shape:", holdout_data.shape)

# Let's see what columns we have after preprocessing
print("\nFirst few columns of preprocessed data:")
print(train_data.columns[:10].tolist())

## Sub-task 4.

Select important variables using the LASSO_LARS linear regression method, going through all possible model complexities within your method and selecting the best one by cross-validation with 5 blocks and MSE as a criterion. In stepwise regression methods, use R-square, p-
value or AIC at your discretion to stop and select the next step. Plot a graph of the CV-MSE dependence on complexity (the number of variables or the number of components in the model), a graph of the trace of standardized
coefficients on complexity. In these graphs, mark the best model complexity by CV with a vertical line.

In [ ]:
# Task 4: Feature Selection with LASSO_LARS (Fixed)

# Define function to convert all remaining categorical features to numeric using one-hot encoding
def prepare_data_for_model(df, target_col="TargetD"):
    # Separate target and features
    y = df[target_col]

    # Get all transformed features and binary indicators
    transformed_features = [
        col
        for col in df.columns
        if "_bc" in col
        or "_target_enc" in col
        or "_threshold_enc" in col
        or "_missing" in col
    ]

    # Get remaining numeric features (exclude target and bins)
    remaining_numeric = [
        col
        for col in df.columns
        if df[col].dtype in ["int64", "float64"]
        and col not in ["TargetB", "TargetD", "TargetD_bins", "ID"]
        and col not in transformed_features
    ]

    # Combine all features
    all_features = transformed_features + remaining_numeric
    X = df[all_features]

    return X, y, all_features


# Prepare data for the model
X_train, y_train, feature_names = prepare_data_for_model(train_data)
X_holdout, y_holdout, _ = prepare_data_for_model(holdout_data)

print(f"Number of features for modeling: {len(feature_names)}")

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_holdout_scaled = scaler.transform(X_holdout)

# Initialize and fit the LASSO_LARS model with cross-validation
# We'll use LassoLarsCV which automatically performs CV to select the best alpha
lasso_lars_cv = LassoLarsCV(cv=5, max_iter=1000)
lasso_lars_cv.fit(X_train_scaled, y_train)

# Get the best complexity (number of non-zero coefficients)
best_complexity = np.sum(lasso_lars_cv.coef_ != 0)
print(f"Best model complexity based on CV: {best_complexity} features")

# Let's take a different approach to analyze complexity vs. MSE
# Using LassoLars directly with different alphas
alphas = np.logspace(-6, 2, 40)  # Generate a range of alphas
complexities = []
cv_mse_values = []

# For each alpha value, fit a model and calculate the complexity and cross-validation MSE

for alpha in alphas:
    model = LassoLars(alpha=alpha, max_iter=1000)
    model.fit(X_train_scaled, y_train)

    # Calculate complexity (number of non-zero coefficients)
    complexity = np.sum(model.coef_ != 0)
    complexities.append(complexity)

    # Calculate cross-validation MSE
    cv_scores = cross_val_score(
        model, X_train_scaled, y_train, cv=5, scoring="neg_mean_squared_error"
    )
    cv_mse = -np.mean(cv_scores)  # Convert negative MSE back to positive
    cv_mse_values.append(cv_mse)

# Print dimensions to verify
print(f"Length of complexities: {len(complexities)}")
print(f"Length of cv_mse_values: {len(cv_mse_values)}")

# Now plot MSE vs. complexity
plt.figure(figsize=(10, 6))
plt.scatter(complexities, cv_mse_values, marker="o", alpha=0.7)

# Instead of using spline interpolation, let's use a simpler approach to show trends
# Group by complexity and average the MSE values
unique_complexities = sorted(set(complexities))
average_mse = []

for complexity in unique_complexities:
    indices = [i for i, c in enumerate(complexities) if c == complexity]
    avg_mse = np.mean([cv_mse_values[i] for i in indices])
    average_mse.append(avg_mse)

# Plot the average MSE per complexity
plt.plot(
    unique_complexities, average_mse, "r-", alpha=0.7, linewidth=2, label="Avg MSE"
)

plt.axvline(
    x=best_complexity,
    color="g",
    linestyle="--",
    label=f"Best Complexity: {best_complexity}",
)
plt.xlabel("Model Complexity (Number of Features)")
plt.ylabel("Cross-Validation MSE")
plt.title("CV-MSE vs. Model Complexity")
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()

# Plot coefficient values vs alpha for the most important features
plt.figure(figsize=(12, 8))

# Identify the top features based on absolute coefficient values
best_model = LassoLars(alpha=lasso_lars_cv.alpha_)
best_model.fit(X_train_scaled, y_train)
top_feature_indices = np.argsort(np.abs(best_model.coef_))[
    -10:
]  # Get indices of top 10 features

# Create a colormap for features


colors = cm.rainbow(np.linspace(0, 1, len(top_feature_indices)))

# Plot coefficient trajectories
for i, feature_idx in enumerate(top_feature_indices):
    feature_name = feature_names[feature_idx]
    coef_values = []

    for alpha in alphas:
        model = LassoLars(alpha=alpha)
        model.fit(X_train_scaled, y_train)
        coef_values.append(model.coef_[feature_idx])

    plt.plot(
        alphas, coef_values, label=feature_name, color=colors[i], linewidth=2, alpha=0.7
    )

plt.axvline(
    x=lasso_lars_cv.alpha_,
    color="k",
    linestyle="--",
    label=f"Best Alpha: {lasso_lars_cv.alpha_:.5f}",
)
plt.xscale("log")
plt.xlabel("Alpha (Regularization Parameter)")
plt.ylabel("Coefficient Values")
plt.title("Top Feature Coefficients vs. Alpha")
plt.grid(True, alpha=0.3)
plt.legend(loc="best", fontsize="small")
plt.show()

# Get the indices of the selected features
selected_features_idx = np.where(lasso_lars_cv.coef_ != 0)[0]
selected_features = [feature_names[i] for i in selected_features_idx]

print(f"Selected {len(selected_features)} features:")
for feature in selected_features:
    print(f"- {feature}")

# Prepare the data with only the selected features
X_train_selected = X_train_scaled[:, selected_features_idx]
X_holdout_selected = X_holdout_scaled[:, selected_features_idx]

# Also create unscaled versions for later use with other models
X_train_selected_unscaled = X_train.iloc[:, selected_features_idx].values
X_holdout_selected_unscaled = X_holdout.iloc[:, selected_features_idx].values

# Train a linear regression model using the selected features
linear_model = LinearRegression()
linear_model.fit(X_train_selected, y_train)

# Calculate R-squared and MSE on both training and holdout sets
train_r2 = linear_model.score(X_train_selected, y_train)
holdout_r2 = linear_model.score(X_holdout_selected, y_holdout)

train_predictions = linear_model.predict(X_train_selected)
holdout_predictions = linear_model.predict(X_holdout_selected)

train_mse = mean_squared_error(y_train, train_predictions)
holdout_mse = mean_squared_error(y_holdout, holdout_predictions)

print(f"Training R-squared: {train_r2:.4f}")
print(f"Holdout R-squared: {holdout_r2:.4f}")
print(f"Training MSE: {train_mse:.2f}")
print(f"Holdout MSE: {holdout_mse:.2f}")

# Calculate the out-of-bag (OOB) error using CV


def calculate_oob_error(model, X, y, cv=5):
    kf = KFold(n_splits=cv, shuffle=True, random_state=42)
    oob_predictions = np.zeros_like(y)

    for train_idx, test_idx in kf.split(X):
        X_cv_train, X_cv_test = X[train_idx], X[test_idx]
        y_cv_train = y.iloc[train_idx] if hasattr(y, "iloc") else y[train_idx]

        model.fit(X_cv_train, y_cv_train)
        oob_predictions[test_idx] = model.predict(X_cv_test)

    return mean_squared_error(y, oob_predictions)


# Calculate OOB error for the linear model
mean_oob_error = calculate_oob_error(LinearRegression(), X_train_selected, y_train)
print(f"Out-of-bag MSE: {mean_oob_error:.2f}")

## Sub-task 5.

For the best selected complexity of the linear model, using bootstrapping (100 bootstrap
samples of 25% of the original size), plot histograms (or kde approximation)
of the distribution of the bias constant in the resulting regression equation (constant b
if regression y=ax+b) indicating the mean value and 95% interval on the graph.
Similarly, estimate the OOB error MSE. How does it compare with the best cross-validation
error and the error on the test part of the sample?

In [ ]:
# Perform bootstrapping to estimate bias constant and OOB error
n_bootstrap = 100
bootstrap_size = int(0.25 * X_train_selected.shape[0])  # 25% of the original size
bootstrap_intercepts = []
bootstrap_oob_errors = []

for i in range(n_bootstrap):
    # Sample with replacement
    bootstrap_indices = np.random.choice(
        X_train_selected.shape[0], size=bootstrap_size, replace=True
    )

    # Identify OOB (out-of-bag) samples
    oob_indices = np.array(
        [i for i in range(X_train_selected.shape[0]) if i not in bootstrap_indices]
    )

    # Training data for this bootstrap sample
    X_bootstrap = X_train_selected[bootstrap_indices]
    y_bootstrap = y_train.iloc[bootstrap_indices]

    # OOB data
    X_oob = X_train_selected[oob_indices]
    y_oob = y_train.iloc[oob_indices]

    # Train the model on the bootstrap sample
    bootstrap_model = LinearRegression()
    bootstrap_model.fit(X_bootstrap, y_bootstrap)

    # Store the intercept (bias constant)
    bootstrap_intercepts.append(bootstrap_model.intercept_)

    # Evaluate on OOB samples and store the MSE
    if len(oob_indices) > 0:  # Make sure there are OOB samples
        oob_predictions = bootstrap_model.predict(X_oob)
        oob_mse = mean_squared_error(y_oob, oob_predictions)
        bootstrap_oob_errors.append(oob_mse)

# Calculate mean and 95% confidence interval for the intercept
mean_intercept = np.mean(bootstrap_intercepts)
ci_lower = np.percentile(bootstrap_intercepts, 2.5)
ci_upper = np.percentile(bootstrap_intercepts, 97.5)

# Calculate mean OOB error
mean_oob_error = np.mean(bootstrap_oob_errors)

# Plot the histogram of bootstrap intercepts
plt.figure(figsize=(10, 6))
plt.hist(bootstrap_intercepts, bins=30, alpha=0.7, color="skyblue", density=True)
plt.axvline(
    mean_intercept, color="red", linestyle="--", label=f"Mean: {mean_intercept:.2f}"
)
plt.axvline(
    ci_lower, color="green", linestyle=":", label=f"95% CI Lower: {ci_lower:.2f}"
)
plt.axvline(
    ci_upper, color="green", linestyle=":", label=f"95% CI Upper: {ci_upper:.2f}"
)
plt.title("Distribution of Regression Intercept (Bias Constant)")
plt.xlabel("Intercept Value")
plt.ylabel("Density")
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()

# Compare OOB error with CV error and holdout error
cv_error = lasso_lars_cv.mse_path_.mean(axis=1)[
    np.argmin(lasso_lars_cv.mse_path_.mean(axis=1))
]
holdout_mse = mean_squared_error(y_holdout, holdout_predictions)

print(f"Bootstrap OOB MSE: {mean_oob_error:.2f}")
print(f"Cross-Validation MSE: {cv_error:.2f}")
print(f"Holdout MSE: {holdout_mse:.2f}")
print(f"Training MSE: {train_mse:.2f}")

# Additional analysis - Compare OOB error distribution with CV error
plt.figure(figsize=(10, 6))
plt.hist(bootstrap_oob_errors, bins=30, alpha=0.7, color="skyblue", density=True)
plt.axvline(
    mean_oob_error,
    color="red",
    linestyle="--",
    label=f"Mean OOB MSE: {mean_oob_error:.2f}",
)
plt.axvline(cv_error, color="green", linestyle=":", label=f"CV MSE: {cv_error:.2f}")
plt.axvline(
    holdout_mse, color="purple", linestyle="-.", label=f"Holdout MSE: {holdout_mse:.2f}"
)
plt.title("Distribution of OOB Mean Squared Error")
plt.xlabel("MSE")
plt.ylabel("Density")
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()

## Sub-task 6.

Use the selected variables to build a nonlinear model for predicting the numerical response using the Poly Regression method, while selecting metaparameters using the HalvingRandomSearchCV method. Note:
- In PLS regressions, use VIP statistics with any threshold in the range [0.5,1] to select variables (after selecting the number of components by cross-validation).
- Note that categorical variables can be either included in the model in their entirety (with all levels) or not.
- For single-layer MLP, you can vary the number of neurons and the regularization constant,
for Poisson Regression, Gamma Regression, and polynomial ridge regression - the regularization constant and the degree of the polynomial (for Gamma and Poisson, use
PolynomialFeatures).

In [ ]:
# Use the selected features to build nonlinear models
# First, prepare unscaled features for the model
X_train_selected_unscaled = X_train.values[:, selected_features_idx]
X_holdout_selected_unscaled = X_holdout.values[:, selected_features_idx]

# Polynomial Regression with Ridge Regularization
poly_ridge_params = {
    "polynomialfeatures__degree": [2, 3],
    "ridge__alpha": np.logspace(-3, 3, 10),
}

poly_ridge_pipeline = Pipeline(
    [
        ("polynomialfeatures", PolynomialFeatures()),
        ("scaler", StandardScaler()),
        ("ridge", Ridge(random_state=42)),
    ]
)

# Use HalvingRandomSearchCV for hyperparameter tuning
halving_cv_poly = HalvingRandomSearchCV(
    poly_ridge_pipeline,
    poly_ridge_params,
    cv=5,
    factor=2,  # Reduce candidates by half each iteration
    random_state=42,
    n_jobs=-1,
    verbose=1,
)

# Fit the model
halving_cv_poly.fit(X_train_selected_unscaled, y_train)

# Get best parameters and score
best_params_poly = halving_cv_poly.best_params_
best_score_poly = halving_cv_poly.best_score_

print("Polynomial Ridge Regression:")
print(f"Best Parameters: {best_params_poly}")
print(f"Best CV Score (negative MSE): {best_score_poly}")

# Evaluate the model on the holdout set
holdout_predictions = halving_cv_poly.predict(X_holdout_selected_unscaled)
holdout_mse_nonlinear = mean_squared_error(y_holdout, holdout_predictions)
holdout_r2_nonlinear = r2_score(y_holdout, holdout_predictions)

print(f"Holdout MSE for Nonlinear Model: {holdout_mse_nonlinear:.2f}")
print(f"Holdout R-squared for Nonlinear Model: {holdout_r2_nonlinear:.2f}")

# Compare with the linear model
print(f"Holdout MSE for Linear Model: {holdout_mse:.2f}")
print(f"Holdout R-squared for Linear Model: {holdout_r2:.2f}")

# Calculate OOB error for the nonlinear model using bootstrapping
bootstrap_oob_errors_nonlinear = []

for i in range(n_bootstrap):
    # Sample with replacement
    bootstrap_indices = np.random.choice(
        X_train_selected_unscaled.shape[0], size=bootstrap_size, replace=True
    )

    # Identify OOB samples
    oob_indices = np.array(
        [
            i
            for i in range(X_train_selected_unscaled.shape[0])
            if i not in bootstrap_indices
        ]
    )

    # OOB data
    X_oob = X_train_selected_unscaled[oob_indices]
    y_oob = y_train.iloc[oob_indices]

    # Evaluate on OOB samples and store the MSE
    if len(oob_indices) > 0:
        oob_predictions = halving_cv_poly.predict(X_oob)
        oob_mse = mean_squared_error(y_oob, oob_predictions)
        bootstrap_oob_errors_nonlinear.append(oob_mse)

# Calculate mean OOB error for the nonlinear model
mean_oob_error_nonlinear = np.mean(bootstrap_oob_errors_nonlinear)

print(f"Bootstrap OOB MSE for Nonlinear Model: {mean_oob_error_nonlinear:.2f}")
print(f"Bootstrap OOB MSE for Linear Model: {mean_oob_error:.2f}")

## Sub-task 7.

Plot a graph - a "lattice" of metaparameter enumeration, indicating the quality of the models with color, and the number of repetitions for halving with the size of the dots). Compare the CV, OOB and holdout of the quality assessment of the obtained linear and nonlinear models, what conclusions can be drawn from this?

In [ ]:
# Extract results from the HalvingRandomSearchCV object for visualization
def extract_param_results(halving_cv, param_names):
    # Get results from all iterations
    results = pd.DataFrame(halving_cv.cv_results_)

    # Extract parameter values and scores
    param_values = {}
    for param in param_names:
        param_values[param] = results[f"param_{param}"].values

    scores = results["mean_test_score"].values
    n_resources = results["n_resources"].values

    return param_values, scores, n_resources


# Create the lattice visualization
def plot_param_lattice(halving_cv, param_names, title):
    param_values, scores, n_resources = extract_param_results(halving_cv, param_names)

    # Create a DataFrame for plotting
    plot_data = pd.DataFrame(
        {
            param_names[0]: param_values[param_names[0]],
            param_names[1]: param_values[param_names[1]],
            "score": scores,
            "n_resources": n_resources,
        }
    )

    # Convert scores to positive values (since CV returns negative MSE)
    if np.all(plot_data["score"] < 0):
        plot_data["score"] = -plot_data["score"]

    # Create the lattice plot
    plt.figure(figsize=(12, 8))

    # Create a scatter plot with color representing score and size representing n_resources
    scatter = plt.scatter(
        x=plot_data[param_names[0]],
        y=plot_data[param_names[1]],
        c=plot_data["score"],
        s=plot_data["n_resources"] / 10,  # Scale down the size for better visualization
        cmap="viridis_r",  # Reversed colormap so darker colors = better scores
        alpha=0.7,
    )

    # Add a colorbar
    cbar = plt.colorbar(scatter)
    cbar.set_label("Mean Test Score (MSE)")

    # Add a legend for the size of points
    unique_resources = sorted(plot_data["n_resources"].unique())
    handles = [
        plt.scatter([], [], s=n / 10, color="gray", alpha=0.7) for n in unique_resources
    ]
    labels = [f"n_resources: {int(n)}" for n in unique_resources]
    plt.legend(handles, labels, title="Halving Iterations", loc="upper right")

    # Set labels and title
    plt.xlabel(param_names[0])
    plt.ylabel(param_names[1])
    plt.title(title)

    # If the parameter is ridge__alpha, use log scale for axis
    if param_names[1] == "ridge__alpha":
        plt.yscale("log")

    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()


# Create the lattice plot for Polynomial Ridge
plot_param_lattice(
    halving_cv_poly,
    ["polynomialfeatures__degree", "ridge__alpha"],
    "Polynomial Ridge Regression: Metaparameter Lattice",
)


# Evaluate the model on various metrics including CV, training and holdout sets
def calculate_oob_error(model, X, y, cv=5):
    """Calculate out-of-bag error using cross-validation"""
    kf = KFold(n_splits=cv, shuffle=True, random_state=42)
    oob_predictions = np.zeros_like(y)

    for train_idx, test_idx in kf.split(X):
        X_cv_train, X_cv_test = X[train_idx], X[test_idx]
        y_cv_train = y[train_idx]

        # Fit the model on training data
        model.fit(X_cv_train, y_cv_train)
        # Predict on test data
        oob_predictions[test_idx] = model.predict(X_cv_test)

    # Calculate MSE
    return mean_squared_error(y, oob_predictions)


# Get the best polynomial ridge model
best_poly_ridge = halving_cv_poly.best_estimator_

# Compute performance metrics
# Cross-validation MSE (from the search)
cv_mse_poly = -halving_cv_poly.best_score_

# Training set performance
y_train_pred_poly = best_poly_ridge.predict(X_train_selected_unscaled)
train_mse_poly = mean_squared_error(y_train, y_train_pred_poly)
train_r2_poly = r2_score(y_train, y_train_pred_poly)

# Holdout set performance
y_holdout_pred_poly = best_poly_ridge.predict(X_holdout_selected_unscaled)
holdout_mse_poly = mean_squared_error(y_holdout, y_holdout_pred_poly)
holdout_r2_poly = r2_score(y_holdout, y_holdout_pred_poly)

# Out-of-bag error using the best model configuration
oob_mse_poly = calculate_oob_error(
    Pipeline(
        [
            (
                "polynomialfeatures",
                PolynomialFeatures(
                    degree=halving_cv_poly.best_params_["polynomialfeatures__degree"],
                    include_bias=False,
                ),
            ),
            (
                "ridge",
                Ridge(
                    alpha=halving_cv_poly.best_params_["ridge__alpha"], random_state=42
                ),
            ),
        ]
    ),
    X_train_selected_unscaled,
    y_train.values if hasattr(y_train, "values") else y_train,
)

# Also calculate metrics for the linear model (from previous tasks) for comparison
# Ensure we have numpy arrays
y_train_array = y_train.values if hasattr(y_train, "values") else y_train
y_holdout_array = y_holdout.values if hasattr(y_holdout, "values") else y_holdout

# Linear model predictions
linear_train_pred = linear_model.predict(X_train_selected)
linear_holdout_pred = linear_model.predict(X_holdout_selected)

# Linear model metrics
linear_train_mse = mean_squared_error(y_train_array, linear_train_pred)
linear_holdout_mse = mean_squared_error(y_holdout_array, linear_holdout_pred)
linear_train_r2 = r2_score(y_train_array, linear_train_pred)
linear_holdout_r2 = r2_score(y_holdout_array, linear_holdout_pred)

# OOB error for linear model
linear_oob_mse = calculate_oob_error(
    LinearRegression(), X_train_selected, y_train_array
)

# Print results in a nice formatted table
print("\nModel Evaluation Results:")
print(f"{'Metric':<20} {'Linear Model':<15} {'Polynomial Ridge':<15}")
print("-" * 50)
print(f"{'CV MSE':<20} {cv_error:<15.4f} {cv_mse_poly:<15.4f}")
print(f"{'OOB MSE':<20} {linear_oob_mse:<15.4f} {oob_mse_poly:<15.4f}")
print(f"{'Training MSE':<20} {linear_train_mse:<15.4f} {train_mse_poly:<15.4f}")
print(f"{'Holdout MSE':<20} {linear_holdout_mse:<15.4f} {holdout_mse_poly:<15.4f}")
print(f"{'Training R²':<20} {linear_train_r2:<15.4f} {train_r2_poly:<15.4f}")
print(f"{'Holdout R²':<20} {linear_holdout_r2:<15.4f} {holdout_r2_poly:<15.4f}")

# Create visual comparison of metrics
metrics = ["CV MSE", "OOB MSE", "Training MSE", "Holdout MSE"]
linear_values = [cv_error, linear_oob_mse, linear_train_mse, linear_holdout_mse]
poly_values = [cv_mse_poly, oob_mse_poly, train_mse_poly, holdout_mse_poly]

# Bar chart comparing MSE metrics
plt.figure(figsize=(12, 6))
x = np.arange(len(metrics))
width = 0.35

plt.bar(x - width / 2, linear_values, width, label="Linear Model")
plt.bar(x + width / 2, poly_values, width, label="Polynomial Ridge")

plt.xlabel("Metrics")
plt.ylabel("Mean Squared Error")
plt.title("MSE Comparison: Linear vs Polynomial Ridge")
plt.xticks(x, metrics)
plt.legend()
plt.grid(True, alpha=0.3)

# Add values on top of bars
for i, v in enumerate(linear_values):
    plt.text(i - width / 2, v + 0.1, f"{v:.2f}", ha="center")
for i, v in enumerate(poly_values):
    plt.text(i + width / 2, v + 0.1, f"{v:.2f}", ha="center")

plt.tight_layout()
plt.show()

# R-squared comparison
metrics_r2 = ["Training R²", "Holdout R²"]
linear_r2_values = [linear_train_r2, linear_holdout_r2]
poly_r2_values = [train_r2_poly, holdout_r2_poly]

plt.figure(figsize=(8, 6))
x_r2 = np.arange(len(metrics_r2))

plt.bar(x_r2 - width / 2, linear_r2_values, width, label="Linear Model")
plt.bar(x_r2 + width / 2, poly_r2_values, width, label="Polynomial Ridge")

plt.xlabel("Metrics")
plt.ylabel("R-squared")
plt.title("R-squared Comparison: Linear vs Polynomial Ridge")
plt.xticks(x_r2, metrics_r2)
plt.legend()
plt.grid(True, alpha=0.3)

# Add values on top of bars
for i, v in enumerate(linear_r2_values):
    plt.text(i - width / 2, v + 0.01, f"{v:.4f}", ha="center")
for i, v in enumerate(poly_r2_values):
    plt.text(i + width / 2, v + 0.01, f"{v:.4f}", ha="center")

plt.tight_layout()
plt.show()

# Plot residuals for both models
plt.figure(figsize=(14, 6))

# Linear model residuals
plt.subplot(1, 2, 1)
linear_residuals = y_holdout_array - linear_holdout_pred
plt.scatter(linear_holdout_pred, linear_residuals, alpha=0.5)
plt.axhline(y=0, color="r", linestyle="-")
plt.xlabel("Predicted Values")
plt.ylabel("Residuals")
plt.title("Linear Model: Residual Plot")
plt.grid(True, alpha=0.3)

# Polynomial Ridge residuals
plt.subplot(1, 2, 2)
poly_residuals = y_holdout_array - y_holdout_pred_poly
plt.scatter(y_holdout_pred_poly, poly_residuals, alpha=0.5)
plt.axhline(y=0, color="r", linestyle="-")
plt.xlabel("Predicted Values")
plt.ylabel("Residuals")
plt.title("Polynomial Ridge: Residual Plot")
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Conclusions
print("\nConclusions:")
print("1. Comparison of CV, OOB, and Holdout Metrics:")
if cv_mse_poly < cv_error:
    print(
        "   - The Polynomial Ridge model shows better cross-validation performance than the linear model."
    )
else:
    print(
        "   - The linear model shows better cross-validation performance than the Polynomial Ridge model."
    )

if oob_mse_poly < linear_oob_mse:
    print(
        "   - The Polynomial Ridge model shows better out-of-bag performance than the linear model."
    )
else:
    print(
        "   - The linear model shows better out-of-bag performance than the Polynomial Ridge model."
    )

if holdout_mse_poly < linear_holdout_mse:
    print(
        "   - The Polynomial Ridge model generalizes better to the holdout set than the linear model."
    )
else:
    print(
        "   - The linear model generalizes better to the holdout set than the Polynomial Ridge model."
    )

# Check for overfitting
linear_diff = abs(linear_holdout_mse - linear_train_mse)
poly_diff = abs(holdout_mse_poly - train_mse_poly)

print("\n2. Overfitting Analysis:")
if poly_diff > linear_diff:
    print(
        f"   - The Polynomial Ridge model shows more evidence of overfitting with a train-holdout MSE difference of {poly_diff:.4f}."
    )
    print(
        f"   - The linear model has a train-holdout MSE difference of {linear_diff:.4f}."
    )
else:
    print(
        f"   - The linear model shows more evidence of overfitting with a train-holdout MSE difference of {linear_diff:.4f}."
    )
    print(
        f"   - The Polynomial Ridge model has a train-holdout MSE difference of {poly_diff:.4f}."
    )
print("\n3. Best Model Based on Performance Metrics:")
if holdout_mse_poly < linear_holdout_mse and holdout_r2_poly > linear_holdout_r2:
    print(
        f"   - The Polynomial Ridge model performs better overall with a holdout MSE of {holdout_mse_poly:.4f} and R² of {holdout_r2_poly:.4f}."
    )
    print(
        f"   - Best parameters: Polynomial degree = {halving_cv_poly.best_params_['polynomialfeatures__degree']}, Alpha = {halving_cv_poly.best_params_['ridge__alpha']:.4f}"
    )
else:
    print(
        f"   - The linear model performs better overall with a holdout MSE of {linear_holdout_mse:.4f} and R² of {linear_holdout_r2:.4f}."
    )

print("\n4. Metaparameter Lattice Analysis:")
print(
    "   - The lattice visualization shows how model performance varies with different combinations of polynomial degree and regularization strength."
)
print("   - Darker colors in the plot represent better model performance (lower MSE).")
print(
    "   - Larger points represent configurations that survived to later rounds of the halving search."
)
print(
    f"   - The optimal configuration found was a polynomial of degree {halving_cv_poly.best_params_['polynomialfeatures__degree']} with alpha = {halving_cv_poly.best_params_['ridge__alpha']:.4f}."
)

print("\n5. Evaluation Methods Comparison:")
print(
    "   - Cross-validation provides a robust estimate of model performance by averaging over multiple train-test splits."
)
print(
    "   - Out-of-bag error gives an unbiased estimate of generalization performance without using a dedicated test set."
)
print(
    "   - Holdout evaluation gives a final assessment of how well the model will perform on completely unseen data."
)

cv_oob_diff_linear = abs(cv_error - linear_oob_mse)
cv_oob_diff_poly = abs(cv_mse_poly - oob_mse_poly)

if cv_oob_diff_linear < cv_oob_diff_poly:
    print(
        f"   - The linear model shows more consistency between CV and OOB metrics (difference: {cv_oob_diff_linear:.4f})"
    )
    print(
        f"   - The Polynomial Ridge model shows a difference of {cv_oob_diff_poly:.4f} between CV and OOB metrics"
    )
else:
    print(
        f"   - The Polynomial Ridge model shows more consistency between CV and OOB metrics (difference: {cv_oob_diff_poly:.4f})"
    )
    print(
        f"   - The linear model shows a difference of {cv_oob_diff_linear:.4f} between CV and OOB metrics"
    )

cv_holdout_diff_linear = abs(cv_error - linear_holdout_mse)
cv_holdout_diff_poly = abs(cv_mse_poly - holdout_mse_poly)

if cv_holdout_diff_linear < cv_holdout_diff_poly:
    print(
        f"   - The linear model shows more consistency between CV and holdout metrics (difference: {cv_holdout_diff_linear:.4f})"
    )
    print(
        f"   - The Polynomial Ridge model shows a difference of {cv_holdout_diff_poly:.4f} between CV and holdout metrics"
    )
else:
    print(
        f"   - The Polynomial Ridge model shows more consistency between CV and holdout metrics (difference: {cv_holdout_diff_poly:.4f})"
    )
    print(
        f"   - The linear model shows a difference of {cv_holdout_diff_linear:.4f} between CV and holdout metrics"
    )

# Visualize predictions vs actual values
plt.figure(figsize=(14, 6))

# Linear model predictions vs actual
plt.subplot(1, 2, 1)
plt.scatter(y_holdout_array, linear_holdout_pred, alpha=0.5)
plt.plot(
    [min(y_holdout_array), max(y_holdout_array)],
    [min(y_holdout_array), max(y_holdout_array)],
    "r--",
)
plt.xlabel("Actual Values")
plt.ylabel("Predicted Values")
plt.title("Linear Model: Actual vs Predicted")
plt.grid(True, alpha=0.3)

# Polynomial Ridge predictions vs actual
plt.subplot(1, 2, 2)
plt.scatter(y_holdout_array, y_holdout_pred_poly, alpha=0.5)
plt.plot(
    [min(y_holdout_array), max(y_holdout_array)],
    [min(y_holdout_array), max(y_holdout_array)],
    "r--",
)
plt.xlabel("Actual Values")
plt.ylabel("Predicted Values")
plt.title("Polynomial Ridge: Actual vs Predicted")
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


# Final summary
print("\nFinal Summary:")
if holdout_mse_poly < linear_holdout_mse:
    print(
        f"The Polynomial Ridge model (degree={halving_cv_poly.best_params_['polynomialfeatures__degree']}, alpha={halving_cv_poly.best_params_['ridge__alpha']:.4f}) outperforms the linear model."
    )
    print(
        f"It achieved a {(1 - holdout_mse_poly/linear_holdout_mse)*100:.2f}% reduction in MSE on the holdout set."
    )
    print(
        f"The improvement in R² is {(holdout_r2_poly - linear_holdout_r2)*100:.2f} percentage points."
    )
else:
    print("The linear model outperforms the Polynomial Ridge model, suggesting that:")
    print("1. The relationship between features and target may be predominantly linear")
    print("2. The added complexity of polynomial features may be causing overfitting")
    print(
        "3. The additional parameters in the polynomial model may not be capturing meaningful patterns"
    )

print("\nRecommendation:")
if holdout_mse_poly < linear_holdout_mse and (train_mse_poly - holdout_mse_poly) < 1.0:
    print(
        "Use the Polynomial Ridge model for future predictions due to its superior performance and reasonable generalization."
    )
elif (
    holdout_mse_poly < linear_holdout_mse and (train_mse_poly - holdout_mse_poly) >= 1.0
):
    print(
        "The Polynomial Ridge model performs better but shows signs of overfitting. Consider:"
    )
    print("1. Collecting more training data")
    print("2. Using a stronger regularization parameter")
    print("3. Reducing the polynomial degree")
else:
    print(
        "Prefer the linear model for its simplicity and better generalization to unseen data."
    )